## <font style="color:lightblue">Header</font>

### <font style="color:lightblue">Imports</font>

In [ ]:
#%load_ext autoreload
#%autoreload 2

import math
import numpy as np
import torch
import torch.nn as nn
from torchinfo import summary
import itertools
import random
import os
from itertools import islice

import sinogap_module_exp as sg



### <font style="color:lightblue">Redefine</font>

In [ ]:
sg.plt.rcParams['figure.dpi']=223
sg.plt.rcParams['axes.spines.left'] = False
sg.plt.rcParams['axes.spines.right'] = False
sg.plt.rcParams['axes.spines.top'] = False
sg.plt.rcParams['axes.spines.bottom'] = False




### <font style="color:lightblue">Configs</font>

In [ ]:
sg.set_seed(7)

sg.TCfg = sg.TCfgClass(
     exec = 0
    ,nofEpochs = None
    ,latentDim = 64
    ,batchSize = 2**6
    ,batchSplit = 2**6
    ,labelSmoothFac = 0.1 # For Fake labels (or set to 0.0 for no smoothing).
    ,learningRateG = 1e-3
    ,learningRateD = 1e-3
    ,num_workers = 16
    ,dataDir = 'data'
)

sg.DCfg = sg.DCfgClass(8,False)


#os.environ["CTAS_MMAP_PATH"] = "/mnt/ssdData/"


### <font style="color:lightblue">Raw Read</font>

In [ ]:
sg.trainSet = sg.createDataSet(sg.TCfg.dataDir, sg.listOfTrainData, expose = 5e-3)
sg.trainLoader = sg.createDataLoader(sg.trainSet, num_workers=sg.TCfg.num_workers)
sg.testSet = sg.createDataSet(sg.TCfg.dataDir+"/test", sg.listOfTestData, exclusive = True)
sg.testLoader = sg.createDataLoader(sg.testSet, num_workers=sg.TCfg.num_workers)

### <font style="color:lightblue">Show</font>

In [ ]:
#st=0.6
sg.examples = [
    ((3, 476, 2880), 0.78),
    ((2, 118,  988), 0.89),
    ((0, 336, 645), 0.725),
    #((0, 298, 1557), 0.846),
    ((3, 925,  592), 0.35)
    #(0, 0, 360, 2108)
    #(0, 0, 338, 773)
    #(0, 0, 411, 1169)
]
#sg.generator.eval()
#print(st)
sg.refImages, sg.refNoises, sg.refBoxes = sg.createReferences(sg.testSet, 0)
#sg.displayImages()
#st += 0.01
#rindex = sg.showMe(sg.testSet)
#img, ridx = sg.testSet[rindex]
#print(ridx, rindex)
#sg.displayImages(img.to(sg.TCfg.device))


## <font style="color:lightblue">Models</font>

### Generator 2pix

In [ ]:
def unrollArch2(gen) :
    gen.encoders =  nn.ModuleList([
        gen.encblock( gen.inChannels/gen.baseChannels,
                           1, 3, norm=False),
        gen.encblock(  1, 2, 3, stride=2),
        gen.encblock(  2, 2, 3),
        gen.encblock(  2, 4, 3, stride=2),
        gen.encblock(  4, 4, 3),
        gen.encblock(  4, 8, 3, stride=2),
        ])
    gen.fcLink = gen.createFClink()
    gen.decoders = nn.ModuleList([
        gen.decblock(16, 4, 3, stride=2),
        gen.decblock( 8, 4, 3),
        gen.decblock( 8, 2, 3, stride=2),
        gen.decblock( 4, 2, 3),
        gen.decblock( 4, 1, 3, stride=2),
        gen.decblock( 2, 1, 3, norm=False),
        ])
    gen.lastTouch = gen.createLastTouch()
    #return gen


class DBricksGenerator2(sg.SubGeneratorTemplate) :
    def __init__(self):
        super(DBricksGenerator2, self).__init__(2, True, batchNorm=True, inChannels=2)
        self.lowResGenerator = None
        self.baseChannels = 64
        unrollArch2(self)
        #sg.load_model(self, model_path="saves/aio/models/dbricks2.pt" )


class StripeGenerator2(sg.SubGeneratorTemplate) :
    def __init__(self):
        super(StripeGenerator2, self).__init__(2, False, batchNorm=True)
        self.lowResGenerator = None
        self.baseChannels = 2
        unrollArch2(self)


class Generator2(sg.GeneratorTemplate) :
    def __init__(self):
        super(Generator2, self).__init__(2, batchNorm=True, inChannels=4)
        self.brickGenerator = DBricksGenerator2()
        self.stripeGenerator = StripeGenerator2()
        self.baseChannels = 4
        unrollArch2(self)
        #sg.load_model(self, model_path="saves/aio/models/full2.pt" )





### Generator 4pix

In [ ]:
def unrollArch4(gen) :
    gen.encoders =  nn.ModuleList([
        gen.encblock( gen.inChannels/gen.baseChannels,
                           1, 3, norm=False),
        gen.encblock(  1, 2, 3, stride=2),
        gen.encblock(  2, 2, 3),
        gen.encblock(  2, 4, 3, stride=2),
        gen.encblock(  4, 4, 3),
        gen.encblock(  4, 8, 3, stride=2),
        gen.encblock(  8, 8, 3),
        gen.encblock(  8,16, 3, stride=2),
        ])
    gen.fcLink = gen.createFClink()
    gen.decoders = nn.ModuleList([
        gen.decblock(32, 8, 3, stride=2),
        gen.decblock(16, 8, 3),
        gen.decblock(16, 4, 3, stride=2),
        gen.decblock( 8, 4, 3),
        gen.decblock( 8, 2, 3, stride=2),
        gen.decblock( 4, 2, 3),
        gen.decblock( 4, 1, 3, stride=2),
        gen.decblock( 2, 1, 3, norm=False),
        ])
    gen.lastTouch = gen.createLastTouch()
    #return gen


class DBricksGenerator4(sg.SubGeneratorTemplate) :
    def __init__(self):
        super(DBricksGenerator4, self).__init__(4, True, batchNorm=True, inChannels=2)
        self.lowResGenerator = DBricksGenerator2()
        #self.lowResGenerator.requires_grad_(False)
        self.baseChannels = 64
        unrollArch4(self)
        #sg.load_model(self, model_path="saves/aio/models/dbricks4.pt" )


class StripeGenerator4(sg.SubGeneratorTemplate) :
    def __init__(self):
        super(StripeGenerator4, self).__init__(4, False, batchNorm=True)
        self.lowResGenerator = None
        self.baseChannels = 2
        unrollArch4(self)


class Generator4(sg.GeneratorTemplate) :
    def __init__(self):
        super(Generator4, self).__init__(4, batchNorm=True, inChannels=4)
        self.brickGenerator = DBricksGenerator4()
        self.brickGenerator.lowResGenerator = None
        self.stripeGenerator = StripeGenerator4()
        self.lowResGenerator = Generator2()
        self.lowResGenerator.requires_grad_(False)
        self.baseChannels = 4
        unrollArch4(self)
        #sg.load_model(self, model_path="saves/aio/gap4_full/model_gen.pt" )


### Generator 8pix

In [ ]:
def unrollArch8(gen) :
    gen.encoders =  nn.ModuleList([
        gen.encblock( gen.inChannels/gen.baseChannels,
                           1, 3, norm=False),
        gen.encblock(  1, 2, 3, stride=2),
        gen.encblock(  2, 2, 3),
        gen.encblock(  2, 4, 3, stride=2),
        gen.encblock(  4, 4, 3),
        gen.encblock(  4, 8, 3, stride=2),
        gen.encblock(  8, 8, 3),
        gen.encblock(  8,16, 3, stride=2),
        gen.encblock( 16,16, 3),
        gen.encblock( 16,32, 3, stride=2),
        ])
    gen.fcLink = gen.createFClink()
    gen.decoders = nn.ModuleList([
        gen.decblock(64,16, 3, stride=2),
        gen.decblock(32,16, 3),
        gen.decblock(32, 8, 3, stride=2),
        gen.decblock(16, 8, 3),
        gen.decblock(16, 4, 3, stride=2),
        gen.decblock( 8, 4, 3),
        gen.decblock( 8, 2, 3, stride=2),
        gen.decblock( 4, 2, 3),
        gen.decblock( 4, 1, 3, stride=2),
        gen.decblock( 2, 1, 3, norm=False),
        ])
    gen.lastTouch = gen.createLastTouch()




class DBricksGenerator8(sg.SubGeneratorTemplate) :
    def __init__(self):
        super(DBricksGenerator8, self).__init__(8, True, batchNorm=True, inChannels=2)
        self.lowResGenerator = None # DBricksGenerator4()
        self.baseChannels = 64
        unrollArch8(self)
        #sg.load_model(self, model_path="saves/aio/models/dbricks8.pt" )




class StripeGenerator8(sg.SubGeneratorTemplate) :
    def __init__(self):
        super(StripeGenerator8, self).__init__(8, False, batchNorm=True)
        self.lowResGenerator = None
        self.baseChannels = 2
        unrollArch8(self)

class FinalGenerator8(sg.SubGeneratorTemplate) :
    def __init__(self):
        super(FinalGenerator8, self).__init__(8, False, batchNorm=True, inChannels=4)
        self.lowResGenerator = None
        self.baseChannels = 4
        self.encoders =  nn.ModuleList([
            self.encblock( self.inChannels/self.baseChannels,
                               1, 3, norm=False),
            self.encblock(  1, 2, 3, stride=2),
            self.encblock(  2, 2, 3),
            self.encblock(  2, 4, 3, stride=2),
            self.encblock(  4, 4, 3),
            self.encblock(  4, 8, 3, stride=2),
            self.encblock(  8, 8, 3),
            self.encblock(  8,16, 3, stride=2),
            self.encblock( 16,16, 3),
            self.encblock( 16,16, 3, stride=2),
            ])
        self.fcLink = self.createFClink()
        self.decoders = nn.ModuleList([
            self.decblock(32,16, 3, stride=2),
            self.decblock(32,16, 3),
            self.decblock(32, 8, 3, stride=2),
            self.decblock(16, 8, 3),
            self.decblock(16, 4, 3, stride=2),
            self.decblock( 8, 4, 3),
            self.decblock( 8, 2, 3, stride=2),
            self.decblock( 4, 2, 3),
            self.decblock( 4, 1, 3, stride=2),
            self.decblock( 2, 1, 3, norm=False),
        ])
        self.lastTouch = self.createLastTouch()





class Generator8(sg.GeneratorTemplate) :
    def __init__(self):
        super(Generator8, self).__init__(8, batchNorm=True, inChannels=4)
        self.brickGenerator = DBricksGenerator8()
        self.stripeGenerator = StripeGenerator8()
        self.lowResGenerator = Generator4()
        self.finalGenerator = FinalGenerator8()
        sg.load_model(self, model_path="tmp8__.pt" )



    def forward(self, images):

        imDevice = images.device

        # channel 0
        with torch.no_grad() :
            preImages = self.preFill(images)
            images = images.clone().detach()
            images[:,[0],*self.cfg.gapRng] = preImages[:,[0],*self.cfg.gapRng].to(imDevice)
        images, norms = sg.normalizeImages(images)
        # channel 1
        stripeImages = self.stripeGenerator.forward( images )
        # channel 2
        bricksIn = torch.cat((      images.view(-1,1, *self.brickGenerator.cfg.sinoSh),
                              stripeImages.view(-1,1, *self.brickGenerator.cfg.sinoSh)),
                             dim=1)
        bricksM = self.brickGenerator.forward(bricksIn).view(-1,1, *self.cfg.sinoSh)
        # channel 3
        edge = self.brickGenerator.cfg.sinoSh[-2]//2
        bricksIn = torch.cat((      images[:,:,edge:-edge,:].reshape(-1,1, *self.brickGenerator.cfg.sinoSh),
                              stripeImages[:,:,edge:-edge,:].reshape(-1,1, *self.brickGenerator.cfg.sinoSh)),
                             dim=1)
        bricksP = images.clone()
        bricksP[:,:,edge:-edge,:] = self.brickGenerator.forward(bricksIn) \
            .view(-1,1, self.cfg.sinoSh[-2]-2*edge, self.cfg.sinoSh[-1] )
        # combine channels and drop in
        modelIn = torch.cat((images,
                             stripeImages.to(imDevice),
                             bricksM.to(imDevice),
                             bricksP.to(imDevice)
                             ), dim=1)
        if self.inChannels > modelIn.shape[1] : # fill missing channels with noise
            #images = images.repeat((1,self.inChannels,1,1))
            lSh = list(modelIn.shape)
            lSh[1] = self.inChannels - lSh[1]
            latentChannels = torch.randn( lSh, device=imDevice )
            modelIn = torch.cat( (modelIn, latentChannels), dim=1 )
        results = self.finalGenerator.forward(modelIn)

        results = sg.reNormalizeImages(results, norms).to(imDevice)
        return results



generator = Generator8()
#generator.requires_grad_(False)
#def reuseLowResWeights(genP, genL) :
#    nofL = sum(1 for dummy in genL.decoders.parameters())
#    nofP = sum(1 for dummy in genP.decoders.parameters())
#    for trg, src in zip( islice(genP.encoders.parameters(), nofL),
#                         genL.encoders.parameters() ) :
#        trg[...] = src[...]
#    for trg, src in zip( islice(genP.decoders.parameters(), nofP-nofL, nofP),
#                         genL.decoders.parameters() ) :
#        trg[...] = src[...]
#    for trg, src in zip( genP.lastTouch.parameters(),
#                         genL.lastTouch.parameters() ) :
#        trg[...] = src[...]
#reuseLowResWeights(generator, generator.lowResGenerator)
#reuseLowResWeights(generator.stripeGenerator, generator.lowResGenerator.stripeGenerator)
#sg.save_model(generator, model_path="tmp.pt" )
#sg.load_model(generator, model_path="tmp.pt" )

#

generator = generator.to(sg.TCfg.device)
sg.generator = generator



model_summary = summary(generator, input_data=[ sg.refImages[[0],...] ] ).__str__()
print(model_summary)
#





### <font style="color:lightblue">Discriminator</font>

In [ ]:

class Discriminator(sg.DiscriminatorTemplate):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.param = nn.Parameter(torch.zeros(1))
    def forward(self, images):
        return torch.zeros((images.shape[0],1), device=sg.TCfg.device)

sg.discriminator = Discriminator()
sg.discriminator = sg.discriminator.to(sg.TCfg.device)
model_summary = summary(sg.discriminator, input_data=sg.refImages[0,...] ).__str__()
print(model_summary)
#sg.writer.add_graph(sg.discriminator, refImages)

sg.optimizer_D = sg.createOptimizer(sg.discriminator, sg.TCfg.learningRateD)



## <font style="color:lightblue">Norms</font>

In [ ]:
#sg.testLoader = sg.createDataLoader(sg.testSet, num_workers=0)
#_ = sg.summarizeMe(sg.testLoader, True)
#_ = sg.summarizeMe(sg.testLoader, False)
#_ = sg.summarizeMe(sg.dataLoader, True)
#_ = sg.summarizeMe(sg.dataLoader, False)
#sg.displayImages()


### no prenorm
sg.EAGLE = sg.Eagle_Loss(patch_size=3, device=sg.TCfg.device, cutoff=0.75)
sg.metrices = {
    'EAGLE'  : sg.Metrics(sg.loss_EAGLE,   8.280e+03, 1/1.1), # 1.219e+03 # cutoff=0.25
    'MSE'    : sg.Metrics(sg.loss_MSE,     3.364e+00, 1/1.6),
    'MSEN'   : sg.Metrics(sg.loss_MSEN,    3.046e+02, 1/7.3),
    'L1L'    : sg.Metrics(sg.loss_L1L,     1.021e+02, 0    ), # 1/1.8
    'L1LN'   : sg.Metrics(sg.loss_L1LN,    1.023e+03, 0    ), # 1/3.5
    'SSIM'   : sg.Metrics(sg.loss_SSIM,    1.111e-04, 1/1.9),
    'MSSSIM' : sg.Metrics(sg.loss_MSSSIM,  3.669e-07, 1/1.9),
    'STD'    : sg.Metrics(sg.loss_STD,     9.792e-03, 1/1.5),
    'COR'    : sg.Metrics(sg.loss_COR,     4.089e-02, 1/4.3),
    'Adv'    : sg.Metrics(sg.loss_Adv_Gen, 0.000e+00, 0),
}
#sg.trackExtremes(False)
#_ = sg.summarizeMe(sg.testLoader, True)

# brick-only estimation
#generator.brickGenerator.lowResGenerator = DBricksGenerator4()
#sg.load_model(generator.brickGenerator.lowResGenerator, model_path="saves/aio/models/dbricks4.pt" )
#generator = generator.to(sg.TCfg.device)
#_ = sg.summarizeMe(sg.testLoader, False)



#sg.displayImages()

## <font style="color:lightblue">Restore checkpoint</font>

In [ ]:
#sg.load_model(sg.generator.lowResGenerator, model_path="full4_catch0.pt" )

#sg.generator.requires_grad_(False)
#sg.generator.lowResGenerator.requires_grad_(False)
#sg.generator.stripeGenerator.requires_grad_(False)
#sg.generator.brickGenerator.requires_grad_ (False)
#sg.generator.finalGenerator.requires_grad_ (False)
#sg.optimizer_G = sg.createOptimizer(sg.generator, sg.TCfg.learningRateG)


sg.generator.stripeGenerator.fcLink.requires_grad_(True)
sg.generator.brickGenerator .fcLink.requires_grad_(True)
sg.generator.finalGenerator .fcLink.requires_grad_(True)
muon_params = [p for p in sg.generator.finalGenerator.fcLink.parameters() if p.ndim == 2] +\
              [p for p in sg.generator.brickGenerator.fcLink.parameters() if p.ndim == 2] +\
              [p for p in sg.generator.stripeGenerator.fcLink.parameters() if p.ndim == 2]
#nonmuon_params = [p for p in sg.generator.finalGenerator.fcLink.parameters() if p.ndim != 2] + \
#                 [p for p in sg.generator.brickGenerator.fcLink.parameters() if p.ndim != 2] +\
#                 [p for p in sg.generator.stripeGenerator.fcLink.parameters() if p.ndim != 2] +\
#                 [p for p in sg.generator.encoders .parameters()] + \
#                 [p for p in sg.generator.decoders .parameters()] + \
#                 [p for p in sg.generator.lastTouch.parameters()] + \
#                 [p for p in sg.generator.brickGenerator.encoders .parameters()] + \
#                 [p for p in sg.generator.brickGenerator.decoders .parameters()] + \
#                 [p for p in sg.generator.brickGenerator.lastTouch.parameters()] + \
#                 [p for p in sg.generator.stripeGenerator.encoders .parameters()] + \
#                 [p for p in sg.generator.stripeGenerator.decoders .parameters()] + \
#                 [p for p in sg.generator.stripeGenerator.lastTouch.parameters()]

sg.optimizer_G = torch.optim.Muon(
        filter(lambda p: p.requires_grad, muon_params),
        lr=sg.TCfg.learningRateG,
        #weight_decay=0.1,
        #momentum=0.95,
        #nesterov=True,
        #ns_coefficients=(3.4445, -4.775, 2.0315),
        #eps=1e-07,
        #ns_steps=5,
        adjust_lr_fn=None)
sg.optimizers_G  = [sg.optimizer_G,]



#model_summary = summary(sg.generator, input_data=[ [sg.refImages[[0],...], sg.refNoises[[0],...]] ] ).__str__()
#print(sg.refImages.shape, sg.refImages[[0],...].shape)
#print(model_summary)


In [ ]:
sg.freeGPUmem()
sg.freeGPUmem()

#sg.optimizer_G = None
#sg.scheduler_G = torch.optim.lr_scheduler.StepLR(sg.optimizer_G, 1, gamma=1)
#sg.scheduler_D = torch.optim.lr_scheduler.StepLR(sg.optimizer_D, 1, gamma=1)

savedCheckPoint = f"checkPoint_{sg.TCfg.exec}"
sg.epoch, sg.imer, sg.minGEpoch, sg.minGLoss, sg.startFrom, sg.resAcc = \
    sg.restoreCheckpoint(savedCheckPoint+"_hourly.pth")
#sg.startFrom, sg.epoch, sg.imer = 0, 0, 0
sg.startFrom = 0
sg.writer = sg.createWriter(sg.TCfg.logDir, True)
#sg.writer.add_graph(sg.generator, ((sg.refImages),) )
#sg.writer.add_graph(sg.discriminator, refImages)
#sg.optimizers_G  = [sg.optimizer_G,]

#sg.load_model(sg.generator, model_path="model_1_gen.pt" )
#sg.load_model(sg.generator.brickGenerator, model_path="saves/complex/models/bricks4.pt" )
#sg.load_model(sg.generator.lowResGenerator, model_path="saves/complex/models/stripe2.pt" )
#sg.generator.brickGenerator.lowResGenerator.requires_grad_(False)
#sg.generator.stripeGenerator.lowResGenerator.requires_grad_(False)


#sg.optimizer_G = sg.createOptimizer(sg.generator, sg.TCfg.learningRateG)


#sg.scheduler_G.gamma = 1-0.01
#sg.scheduler_D.gamma = 1-0.005
#sg.optimizer_G.step()
#torch.optim.lr_scheduler.LambdaLR(sg.optimizer_G, lambda epoch: 1).step()
#sg.optimizer_D.step()
#torch.optim.lr_scheduler.LambdaLR(sg.optimizer_D, lambda epoch: 0.05).step()

if sg.scheduler_G is not None:
    print(sg.epoch, sg.imer)
    print(f"Initial LR : G {sg.scheduler_G.get_last_lr()[0]:.3e} ({sg.scheduler_G.get_last_lr()[0]/sg.TCfg.learningRateG:.4f})")
                  #f" D {sg.scheduler_D.get_last_lr()[0]:.3e} ({sg.scheduler_D.get_last_lr()[0]/sg.TCfg.learningRateD:.4f}), Epoch {sg.epoch} {sg.imer}")


### <font style="color:lightblue">Optimizers</font>

In [ ]:
#sg.load_model(sg.generator.lowResGenerator, model_path="full4_catch0.pt" )

sg.generator.requires_grad_(False)
sg.generator.lowResGenerator.requires_grad_(False)
sg.generator.stripeGenerator.requires_grad_(False)
sg.generator.brickGenerator.requires_grad_ (False)
sg.generator.finalGenerator.requires_grad_ (False)
#sg.optimizer_G = sg.createOptimizer(sg.generator, sg.TCfg.learningRateG)


sg.generator.stripeGenerator.fcLink.requires_grad_(True)
sg.generator.brickGenerator .fcLink.requires_grad_(True)
sg.generator.finalGenerator .fcLink.requires_grad_(True)
muon_params = [p for p in sg.generator.finalGenerator.fcLink.parameters() if p.ndim == 2] +\
              [p for p in sg.generator.brickGenerator.fcLink.parameters() if p.ndim == 2] +\
              [p for p in sg.generator.stripeGenerator.fcLink.parameters() if p.ndim == 2]
#nonmuon_params = [p for p in sg.generator.finalGenerator.fcLink.parameters() if p.ndim != 2] + \
#                 [p for p in sg.generator.brickGenerator.fcLink.parameters() if p.ndim != 2] +\
#                 [p for p in sg.generator.stripeGenerator.fcLink.parameters() if p.ndim != 2] +\
#                 [p for p in sg.generator.encoders .parameters()] + \
#                 [p for p in sg.generator.decoders .parameters()] + \
#                 [p for p in sg.generator.lastTouch.parameters()] + \
#                 [p for p in sg.generator.brickGenerator.encoders .parameters()] + \
#                 [p for p in sg.generator.brickGenerator.decoders .parameters()] + \
#                 [p for p in sg.generator.brickGenerator.lastTouch.parameters()] + \
#                 [p for p in sg.generator.stripeGenerator.encoders .parameters()] + \
#                 [p for p in sg.generator.stripeGenerator.decoders .parameters()] + \
#                 [p for p in sg.generator.stripeGenerator.lastTouch.parameters()]

sg.optimizer_G = torch.optim.Muon(
        filter(lambda p: p.requires_grad, muon_params),
        lr=sg.TCfg.learningRateG,
        #weight_decay=0.1,
        #momentum=0.95,
        #nesterov=True,
        #ns_coefficients=(3.4445, -4.775, 2.0315),
        #eps=1e-07,
        #ns_steps=5,
        adjust_lr_fn=None)
sg.optimizers_G  = [sg.optimizer_G,]



#model_summary = summary(sg.generator, input_data=[ [sg.refImages[[0],...], sg.refNoises[[0],...]] ] ).__str__()
#print(sg.refImages.shape, sg.refImages[[0],...].shape)
#print(model_summary)


### <font style="color:lightblue">Display</font>

In [ ]:
#sg.generator.brickGenerator.lowResGenerator = None
sg.generator.eval()
_ = sg.displayImages()
#_ = sg.summarizeMe(sg.testLoader, False)


## <font style="color:lightblue">Execute</font>

In [ ]:
sg.freeGPUmem()
sg.freeGPUmem()

#torch.optim.lr_scheduler.LambdaLR(sg.optimizers_G[0], lambda epoch: 1e-1).step()
sg.scheduler_G = torch.optim.lr_scheduler.StepLR(sg.optimizers_G[0], 1, gamma=1-0.005)
def my_beforeReport(sglocals) :
    #return
    message = ""

    message += "G" + sg.adjustScheduler(sg.scheduler_G, sg.TCfg.learningRateG, 1e-4)
    sg.writer.add_scalars("Aux", {'LrnRate': sg.scheduler_G.get_last_lr()[0]/sg.TCfg.learningRateG}, sg.imer)

    #for metrics in ('L1L','SSIM','MSSSIM') :
    #    sg.metrices[metrics].weight = max(0, sg.metrices[metrics].weight - 0.005)
    #    message += f"{metrics} weight: {sg.metrices[metrics].weight}. "
    #    sg.writer.add_scalars("Aux", {metrics: sg.metrices[metrics].weight}, sg.imer)

    print(message)
sg.beforeReport = my_beforeReport


def my_beforeEachEpoch(sglocals) :
    sg.generator.eval()
    return
    sg.TCfg.batchSize = min(2**12, sg.TCfg.batchSize * 2)
    sg.TCfg.batchSplit = max(2**0, int(sg.TCfg.batchSize / 2**2))
    sg.writer.add_scalars("Aux", {'Batch size': math.log(sg.TCfg.batchSize,2)}, sg.imer)
    print(f"Batch size: {sg.TCfg.batchSize}, split: {sg.TCfg.batchSplit}", )
sg.beforeEachEpoch = my_beforeEachEpoch

#sg.TCfg.num_workers=0
try :
    sg.train(savedCheckPoint)
except :
    #del sg.dataLoader
    #del sg.testLoaderhuiVam
    sg.freeGPUmem()
    1/10 # to release Jupyuter memory in the next step
    sg.epoch -= 1
    raise



# bricks pretrained



## <font style="color:lightblue">Post</font>

In [ ]:
sg.saveCheckPoint(savedCheckPoint+".pth",
                           sg.epoch+1, sg.imer, sg.minGEpoch, sg.minGLoss,
                           sg.generator, sg.discriminator,
                           sg.optimizer_G, sg.optimizer_D)


In [ ]:
for key in sg.resAcc.metrices.keys() :
    if sg.metrices[key].norm > 0 :
        sg.writer.add_scalars("Metrices_per_epoch", {key : sg.resAcc.metrices[key],}, sg.epoch )

resTest = sg.summarizeMe(sg.testLoader, False)
resTest *= 1/resTest.nofIm
sg.writer.add_scalars("Losses epoch test",
                   {'Dis': resTest.lossD
                   ,'Gen': resTest.lossG
                   #,'Adv' : resTest.metrices['Adv']
                   }, sg.epoch )
for key in resTest.metrices.keys() :
    if sg.metrices[key].norm > 0 :
        sg.writer.add_scalars("Metrices epoch test", {key : resTest.metrices[key],}, sg.epoch )
sg.writer.add_scalars("Probs epoch test",
                   {'Ref':resTest.predReal
                   ,'Gen':resTest.predFake
                   #,'Pre':trainRes.predGen
                   }, sg.epoch )





In [ ]:
sg.testMe(trainSet, 5)

### <font style="color:lightblue">Save results</font>

In [ ]:
sg.save_model(generator.stripeGenerator, "tmp_stripe.pt")